In [40]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from datetime import datetime, timedelta
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import intercluster_distance
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from wordcloud import WordCloud

### (1) 데이터 불러오기

In [ ]:
DATA_PATH = ""
os.chdir(DATA_PATH)
df = pd.read_csv(DATA_PATH+"tiktoker_crawling_df_0127.csv")

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1680 non-null   int64 
 1   name        1680 non-null   object
 2   follower    1680 non-null   object
 3   view        1680 non-null   object
 4   like        1680 non-null   object
 5   comment     1680 non-null   int64 
 6   save        1680 non-null   object
 7   date        1680 non-null   object
 8   info_tag    1664 non-null   object
 9   hash_tag    1680 non-null   object
dtypes: int64(2), object(8)
memory usage: 131.4+ KB


### (2)전처리

#### 전처리_5 : 숫자형, 날짜형 컬럼
> - view, like, comment, save 컬럼 : 단위(k,m) 변환
> - date
    >> - 25년도 1월 업로드한 경우 2025-01-nn으로 변환 
    >> - nh ago, nd ago, 1w ago인 경우 2025-01-21기준으로 날짜 계산

##### 5-1. date 컬럼 형식 통일 및 datetime으로 변환

In [43]:
def preprocess_date(df):
    today = datetime(2025, 1, 21)

    for i in range(len(df)):
    
        if len(df['date'][i]) >= 8:
            df.loc[i, 'upload_date'] = df['date'][i]
    
        elif len(df['date'][i]) >= 6:
            date = df['date'][i].replace(' ','')
            alphabet_list = [a for a in date]
    
            if 'h' in alphabet_list:
                df.loc[i, 'upload_date'] = today
    
            elif 'd' in alphabet_list:
                day = alphabet_list[0]
                day = int(day)
                df.loc[i, 'upload_date'] = today - timedelta(days=day)
    
            elif 'w' in alphabet_list:
                week = alphabet_list[0]
                week = int(week)
                df.loc[i, 'upload_date'] = today - timedelta(weeks=week)
    
        elif len(df['date'][i]) == 4:
            df.loc[i, 'upload_date'] = '2025-' + df['date'][i]
    
        elif len(df['date'][i]) == 3:
            df.loc[i, 'upload_date'] = '2025-' + df['date'][i]

    # datetime형으로 변환
    df['upload_date'] = pd.to_datetime(df['upload_date'], format="%Y-%m-%d", errors='coerce')


preprocess_date(df)

In [44]:
# 결과 확인 1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   1680 non-null   int64         
 1   name         1680 non-null   object        
 2   follower     1680 non-null   object        
 3   view         1680 non-null   object        
 4   like         1680 non-null   object        
 5   comment      1680 non-null   int64         
 6   save         1680 non-null   object        
 7   date         1680 non-null   object        
 8   info_tag     1664 non-null   object        
 9   hash_tag     1680 non-null   object        
 10  upload_date  1680 non-null   datetime64[us]
dtypes: datetime64[us](1), int64(2), object(8)
memory usage: 144.5+ KB


#### view, like, comment, save컬럼 단위 변환

In [45]:
# follower 컬럼 단위 전처리

def process_follower(df):
    for i in range(len(df)):
        follower = df['follower'][i].replace(' ','')
        follower_list = [a for a in follower]
        if 'K' in follower_list:
            num = follower.split('K')[0]
            num = float(num)
            follower_cal = num*1000
            df.loc[i, 'follower_cnt'] = follower_cal
        elif 'M' in follower_list:
            num = follower.split('M')[0]
            num = float(num)
            follower_cal = num*1000000
            df.loc[i, 'follower_cnt'] = follower_cal
        else:
            num = df['follower'][i].replace(' ','')
            follower_cal = float(num)
            df.loc[i, 'follower_cnt'] = follower_cal

process_follower(df)

In [46]:
# view 컬럼 단위 전처리

def process_view(df):
    for i in range(len(df)):
        view = df['view'][i].replace(' ','')
        view_list = [a for a in view]
        if 'K' in view_list:
            num = view.split('K')[0]
            num = float(num)
            view_cal = num*1000
            df.loc[i, 'view_cnt'] = view_cal
        elif 'M' in view_list:
            num = view.split('M')[0]
            num = float(num)
            view_cal = num*1000000
            df.loc[i, 'view_cnt'] = view_cal
        else:
            num = df['view'][i].replace(' ','')
            view_cal = float(num)
            df.loc[i, 'view_cnt'] = view_cal

process_view(df)

In [47]:
# like 컬럼 단위 전처리

def process_like(df):
    for i in range(len(df)):
        like = df['like'][i].replace(' ','')
        like_list = [a for a in like]
        if 'K' in like_list:
            num = like.split('K')[0]
            num = float(num)
            like_cal = num*1000
            df.loc[i, 'like_cnt'] = like_cal
        elif 'M' in like_list:
            num = like.split('M')[0]
            num = float(num)
            like_cal = num*1000000
            df.loc[i, 'like_cnt'] = like_cal
        else:
            num = df['like'][i].replace(' ','')
            like_cal = float(num)
            df.loc[i, 'like_cnt'] = like_cal

process_like(df)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   float64       
 13  like_cnt      1680 non-null   float64       
dtypes: datetime64[us](1), float64(3), int64(2), object(8)
memory usage: 183.9+ KB


In [49]:
# comment 컬럼 단위 전처리

def process_comment(df):
    for i in range(len(df)):
        comment = str(df['comment'][i]).replace(' ','')
        comment_list = [a for a in comment]
        if 'K' in comment_list:
            num = comment.split('K')[0]
            num = float(num)
            comment_cal = num*1000
            df.loc[i, 'comment_cnt'] = comment_cal
        elif 'M' in comment_list:
            num = comment.split('M')[0]
            num = float(num)
            comment_cal = num*1000000
            df.loc[i, 'comment_cnt'] = comment_cal
        else:
            num = str(df['comment'][i]).replace(' ','')
            comment_cal = float(num)
            df.loc[i, 'comment_cnt'] = comment_cal

process_comment(df)

In [50]:
# save 컬럼 단위 전처리

def process_save(df):
    for i in range(len(df)):
        save = df['save'][i].replace(' ','')
        save_list = [a for a in save]
        if 'K' in save_list:
            num = save.split('K')[0]
            num = float(num)
            save_cal = num*1000
            df.loc[i, 'save_cnt'] = save_cal
        elif 'M' in save_list:
            num = save.split('M')[0]
            num = float(num)
            save_cal = num*1000000
            df.loc[i, 'save_cnt'] = save_cal
        else:
            num = df['save'][i].replace(' ','')
            save_cal = float(num)
            df.loc[i, 'save_cnt'] = save_cal

process_save(df)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   float64       
 13  like_cnt      1680 non-null   float64       
 14  comment_cnt   1680 non-null   float64       
 15  save_cnt      1680 non-null   float64 

In [52]:
# view_cnt, like_cnt, comment_cnt, save_cnt컬럼 float에서 int로 변환

df['view_cnt'] = df['view_cnt'].round(0).astype(int)
df['like_cnt'] = df['like_cnt'].round(0).astype(int)
df['comment_cnt'] = df['comment_cnt'].round(0).astype(int)
df['save_cnt'] = df['save_cnt'].round(0).astype(int)

In [53]:
# 결과 확인 1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   int64         
 13  like_cnt      1680 non-null   int64         
 14  comment_cnt   1680 non-null   int64         
 15  save_cnt      1680 non-null   int64   

In [54]:
# 결과 확인 2
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   int64         
 13  like_cnt      1680 non-null   int64         
 14  comment_cnt   1680 non-null   int64         
 15  save_cnt      1680 non-null   int64   

In [55]:
df.columns

Index(['Unnamed: 0', 'name', 'follower', 'view', 'like', 'comment', 'save',
       'date', 'info_tag', 'hash_tag', 'upload_date', 'follower_cnt',
       'view_cnt', 'like_cnt', 'comment_cnt', 'save_cnt'],
      dtype='object')

In [56]:
# 필요한 컬럼만 추출

df = df[['name', 'follower_cnt', 'upload_date', 'view_cnt', 'like_cnt','comment_cnt', 'save_cnt', 'info_tag']]

In [57]:
df

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag
0,mydelicate,"399,800.000",2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...
1,mydelicate,"399,800.000",2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...
2,mydelicate,"399,800.000",2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...
3,mydelicate,"399,800.000",2025-01-14,22100,4206,193,602,new Flower Knows strawberry cupid collection u...
4,mydelicate,"399,800.000",2025-01-18,17200,2995,123,432,my favourite products for soft skin 🎀💓 names:\...
...,...,...,...,...,...,...,...,...
1675,amyflamy1,"4,300,000.000",2024-12-14,1400000,131400,640,3686,My hands be too smoll\n#fyp #dance #dancechall...
1676,amyflamy1,"4,300,000.000",2024-12-13,3800000,431000,1654,21100,We single for a reason✨💅\n\n#fyp #dance #dance...
1677,amyflamy1,"4,300,000.000",2024-12-12,6200000,647700,3055,33400,Bibim noodels mukbang🍜\n#mukbang #mukbangeatin...
1678,amyflamy1,"4,300,000.000",2024-12-10,936900,94400,536,3589,Welcome girls you’re safe here\n#gurwm #gurwmr...


In [58]:
df

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag
0,mydelicate,"399,800.000",2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...
1,mydelicate,"399,800.000",2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...
2,mydelicate,"399,800.000",2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...
3,mydelicate,"399,800.000",2025-01-14,22100,4206,193,602,new Flower Knows strawberry cupid collection u...
4,mydelicate,"399,800.000",2025-01-18,17200,2995,123,432,my favourite products for soft skin 🎀💓 names:\...
...,...,...,...,...,...,...,...,...
1675,amyflamy1,"4,300,000.000",2024-12-14,1400000,131400,640,3686,My hands be too smoll\n#fyp #dance #dancechall...
1676,amyflamy1,"4,300,000.000",2024-12-13,3800000,431000,1654,21100,We single for a reason✨💅\n\n#fyp #dance #dance...
1677,amyflamy1,"4,300,000.000",2024-12-12,6200000,647700,3055,33400,Bibim noodels mukbang🍜\n#mukbang #mukbangeatin...
1678,amyflamy1,"4,300,000.000",2024-12-10,936900,94400,536,3589,Welcome girls you’re safe here\n#gurwm #gurwmr...


### 전처리_2 : 해시태그 컬럼 생성
> - 크롤링 한 데이터를 csv파일로 변환하는 과정에서 hash_tag컬럼의 일부 행들이 손상되는 문제 발생
> - info 컬럼에서 해시태그 추출하여 해시태그 컬럼 다시 생성

#### info 컬럼에서 해시태그 추출

In [59]:
def new_hash_tag_process(df):
    for i in range(len(df)):
        info = str(df['info_tag'][i])
        info_split_list = info.split(' ')
        hash_tag_list = []
        for a in info_split_list:
            if a.startswith('#'):
                hash_tag_list.append(a)
            elif a.startswith('@'):
                hash_tag_list.append(a)
            else:
                pass
        df.loc[i, 'hash_tag'] = ','.join(hash_tag_list)

new_hash_tag_process(df)

In [60]:
# 결과 확인
df.head(3)

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,"399,800.000",2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,"399,800.000",2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,"399,800.000",2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."


In [61]:
# 결과 확인 2
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          1680 non-null   object        
 1   follower_cnt  1680 non-null   float64       
 2   upload_date   1680 non-null   datetime64[us]
 3   view_cnt      1680 non-null   int64         
 4   like_cnt      1680 non-null   int64         
 5   comment_cnt   1680 non-null   int64         
 6   save_cnt      1680 non-null   int64         
 7   info_tag      1664 non-null   object        
 8   hash_tag      1680 non-null   object        
dtypes: datetime64[us](1), float64(1), int64(4), object(3)
memory usage: 118.3+ KB


In [ ]:

df.to_csv('tiktoker_final_df_0127.csv',index=False, encoding="utf-8-sig")

### 전처리_4 : 자연어 처리

In [ ]:
# 데이터 다시 불러오기 
df = pd.read_csv('tiktoker_final_df_0127.csv')

In [80]:
df.head(3)

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,"399,800.000",2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,"399,800.000",2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,"399,800.000",2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."


In [81]:
# nltk 에서 Punkt tokenizer & stopwords list를 다운로드
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /Users/jun/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### EDA

#### 인플루언서 사이즈별 광고가격 분류
- 나노 인플루언서 (1,000–10,000 팔로워): 게시물당 $5~$25
- 마이크로 인플루언서 (10,000–100,000 팔로워): 게시물당 $25~$125
- 중간급 인플루언서 (100,000–500,000 팔로워): 게시물당 $125~$1,200
- 매크로 인플루언서 (500,000–1백만 팔로워): 게시물당 $1,200~$2,500
- 메가 인플루언서 (1백만+ 팔로워): 게시물당 $2,500 이상

광고 가격 분류
평균 영상 조회 수 * 2만

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          1680 non-null   object 
 1   follower_cnt  1680 non-null   float64
 2   upload_date   1680 non-null   object 
 3   view_cnt      1680 non-null   int64  
 4   like_cnt      1680 non-null   int64  
 5   comment_cnt   1680 non-null   int64  
 6   save_cnt      1680 non-null   int64  
 7   info_tag      1664 non-null   object 
 8   hash_tag      1620 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 118.3+ KB


#### 인플루언서별 평균 view, like, comment, save

In [83]:
# 소수점 이하 3자리까지 출력되도록 설정
pd.options.display.float_format = '{:,.3f}'.format

mean_df = df.groupby('name').agg({'follower_cnt':'mean',
                                    'view_cnt':'mean',
                                    'like_cnt':'mean',
                                    'comment_cnt':'mean',
                                    'save_cnt':'mean'
                                    }).reset_index()

In [84]:
mean_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt
0,.blissdiaries,"81,700.000","9,717.967","1,447.333",60.467,196.200
1,.m.egan,"238,200.000","178,070.000","23,864.300",182.533,"3,527.300"
2,.woniluv,"135,600.000","41,956.200","2,399.000",85.767,369.933
3,4chriisty,"79,500.000","26,122.500","4,119.733",74.133,385.367
4,__.annie.s,"37,300.000","10,332.733","2,375.100",128.467,465.100
5,_luvwonsnnx,"23,900.000","8,783.467","1,593.633",57.867,261.967
6,aeiezo.pdf,"21,400.000","20,647.133","2,587.733",95.900,350.233
7,ameenbeautytips,"4,800,000.000","418,445.967","1,602.867",14.767,143.100
8,amyflamy1,"4,300,000.000","2,490,070.000","272,720.000","1,466.933","19,117.433"
9,asiamelon,"13,700.000","1,751.233",164.300,12.267,19.767


In [85]:
mean_df.loc[:,'ad_cost'] = mean_df['view_cnt'] * 20000

In [87]:
mean_df.iloc[1:, 1:] = mean_df.iloc[1:, 1:].replace(",", "", regex=True).astype(float).round(0).astype(int)

In [88]:
# erp
mean_df['ER%'] = (mean_df['like_cnt']+mean_df['comment_cnt'])/mean_df['follower_cnt'] * 100 

In [89]:
mean_df

,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%
1,238200,178070,23864,183,3527,3561400000,10.095
2,135600,41956,2399,86,370,839124000,1.833
3,79500,26122,4120,74,385,522450000,5.275
4,37300,10333,2375,128,465,206654667,6.710
5,23900,8783,1594,58,262,175669333,6.912
6,21400,20647,2588,96,350,412942667,12.542
7,4800000,418446,1603,15,143,8368919333,0.034
8,4300000,2490070,272720,1467,19117,49801400000,6.376
9,13700,1751,164,12,20,35024667,1.285
10,78900,28750,1746,9,266,574992000,2.224


In [90]:
# 업로드 빈도
df[['name','upload_date']]

,name,upload_date
0,mydelicate,2025-01-21
1,mydelicate,2025-01-18
2,mydelicate,2025-01-15
3,mydelicate,2025-01-14
4,mydelicate,2025-01-18
...,...,...
1675,amyflamy1,2024-12-14
1676,amyflamy1,2024-12-13
1677,amyflamy1,2024-12-12
1678,amyflamy1,2024-12-10


In [91]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

# ✅ 현재 날짜 기준
TODAY = datetime(2025, 1, 27)  # 현재 날짜 설정

df['upload_date'] = pd.to_datetime(df['upload_date'])  # 날짜형으로 변환

# ✅ name 별로 업로드 간격 계산
tiktoker_df = df.sort_values(by=["name", "upload_date"])  # 이름 + 날짜순 정렬
tiktoker_df["prev_date"] = tiktoker_df.groupby("name")["upload_date"].shift(1)  # 이전 영상 날짜
tiktoker_df["upload_gap"] = (tiktoker_df["upload_date"] - tiktoker_df["prev_date"]).dt.days  # 간격(일)

# ✅ 평균 업로드 주기 계산
upload_cycle = tiktoker_df.groupby("name")["upload_gap"].mean().reset_index()
upload_cycle.rename(columns={"upload_gap": "avg_upload_interval"}, inplace=True)

# ✅ 결과 출력
print(upload_cycle)

                 name  avg_upload_interval
0       .blissdiaries                4.069
1             .m.egan                5.034
2            .woniluv                2.379
3           4chriisty                2.483
4          __.annie.s                3.138
5         _luvwonsnnx                1.483
6          aeiezo.pdf                3.483
7     ameenbeautytips                0.172
8           amyflamy1                1.448
9           asiamelon                3.345
10           ayheyt3u                5.759
11         aylennpark                1.621
12         bwnniewony                1.931
13   caspertheghostyy                0.828
14     cleangirlhacks                3.034
15   cleanlivingkarly                0.552
16         danicolexx                0.310
17         dearwonii_                2.655
18             emchu_                3.414
19          fromh1ae_                2.207
20            gae4hrv                3.586
21     glowupwithinah                0.241
22         

In [92]:
df

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,"399,800.000",2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,"399,800.000",2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,"399,800.000",2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."
3,mydelicate,"399,800.000",2025-01-14,22100,4206,193,602,new Flower Knows strawberry cupid collection u...,"#flowerknows,#unboxing,#cbeauty,#chinesemakeup..."
4,mydelicate,"399,800.000",2025-01-18,17200,2995,123,432,my favourite products for soft skin 🎀💓 names:\...,"@cosmetic_jolse,@YesStyle,@sephora,@Sol,#selfc..."
...,...,...,...,...,...,...,...,...,...
1675,amyflamy1,"4,300,000.000",2024-12-14,1400000,131400,640,3686,My hands be too smoll\n#fyp #dance #dancechall...,"#dance,#dancechallenge"
1676,amyflamy1,"4,300,000.000",2024-12-13,3800000,431000,1654,21100,We single for a reason✨💅\n\n#fyp #dance #dance...,"#dance,#dancechallenge,#dresstoimpress,#fashio..."
1677,amyflamy1,"4,300,000.000",2024-12-12,6200000,647700,3055,33400,Bibim noodels mukbang🍜\n#mukbang #mukbangeatin...,"#mukbangeatingshow,#mukbangvideo,#eating,#eati..."
1678,amyflamy1,"4,300,000.000",2024-12-10,936900,94400,536,3589,Welcome girls you’re safe here\n#gurwm #gurwmr...,"#gurwmroutine,#grwm,#grwmroutine,#skincarerout..."


In [93]:
upload_cycle

,name,avg_upload_interval
0,.blissdiaries,4.069
1,.m.egan,5.034
2,.woniluv,2.379
3,4chriisty,2.483
4,__.annie.s,3.138
5,_luvwonsnnx,1.483
6,aeiezo.pdf,3.483
7,ameenbeautytips,0.172
8,amyflamy1,1.448
9,asiamelon,3.345


In [186]:
mean_df['avg_upload_interval'] = upload_cycle['avg_upload_interval']
mean_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,tiktoker_size,ad_cost,ER,avg_upload_interval
0,.blissdiaries,"81,700.00","9,717.97","1,447.33",60.47,196.20,micro_influancer,25.00,1.85,4.07
1,.m.egan,"238,200.00","178,070.00","23,864.30",182.53,"3,527.30",middle_influancer,125.00,10.10,5.03
2,.woniluv,"135,600.00","41,956.20","2,399.00",85.77,369.93,middle_influancer,125.00,1.83,2.38
3,4chriisty,"79,500.00","26,122.50","4,119.73",74.13,385.37,micro_influancer,25.00,5.28,2.48
4,__.annie.s,"37,300.00","10,332.73","2,375.10",128.47,465.10,micro_influancer,25.00,6.71,3.14
5,_luvwonsnnx,"23,900.00","8,783.47","1,593.63",57.87,261.97,micro_influancer,25.00,6.91,1.48
6,aeiezo.pdf,"21,400.00","20,647.13","2,587.73",95.90,350.23,micro_influancer,25.00,12.54,3.48
7,ameenbeautytips,"4,800,000.00","418,445.97","1,602.87",14.77,143.10,mega_influancer,"2,500.00",0.03,0.17
8,amyflamy1,"4,300,000.00","2,490,070.00","272,720.00","1,466.93","19,117.43",mega_influancer,"2,500.00",6.38,1.45
9,asiamelon,"13,700.00","1,751.23",164.30,12.27,19.77,micro_influancer,25.00,1.29,3.34


In [142]:
top5_hash_tag_df = pd.read_csv('/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130/tiktoker_top5_hashtags.csv')

In [143]:
mean_df = mean_df.merge(top5_hash_tag_df, on='name', how='left')
mean_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,tiktoker_size,ad_cost,ER,avg_upload_interval,hash_tag_top5
0,.blissdiaries,"81,700.00","9,717.97","1,447.33",60.47,196.20,micro_influancer,25.00,1.85,4.07,"['kpopfyp', 'douyin', 'haul', 'xyzabc', 'kbeau..."
1,.m.egan,"238,200.00","178,070.00","23,864.30",182.53,"3,527.30",middle_influancer,125.00,10.10,5.03,"['beautyinfluencer', 'jolse', 'xiaohongshu', '..."
2,.woniluv,"135,600.00","41,956.20","2,399.00",85.77,369.93,middle_influancer,125.00,1.83,2.38,"['tiktokban', 'tiktokbanned', 'pretty', 'dance..."
3,4chriisty,"79,500.00","26,122.50","4,119.73",74.13,385.37,micro_influancer,25.00,5.28,2.48,"['jbeauty', 'blue', 'romand', 'pr', 'palette']"
4,__.annie.s,"37,300.00","10,332.73","2,375.10",128.47,465.10,micro_influancer,25.00,6.71,3.14,"['influencer', 'abib', 'influencers', 'minivlo..."
5,_luvwonsnnx,"23,900.00","8,783.47","1,593.63",57.87,261.97,micro_influancer,25.00,6.91,1.48,"['foruu', 'dontletthisflop', 'fyppp', 'viralvi..."
6,aeiezo.pdf,"21,400.00","20,647.13","2,587.73",95.90,350.23,micro_influancer,25.00,12.54,3.48,"['moot', 'creatorsearchinsights', 'routine', '..."
7,ameenbeautytips,"4,800,000.00","418,445.97","1,602.87",14.77,143.10,mega_influancer,"2,500.00",0.03,0.17,['ameenbeautytips']
8,amyflamy1,"4,300,000.00","2,490,070.00","272,720.00","1,466.93","19,117.43",mega_influancer,"2,500.00",6.38,1.45,"['gurwm', 'gurwmroutine', 'grwmroutine', 'make..."
9,asiamelon,"13,700.00","1,751.23",164.30,12.27,19.77,micro_influancer,25.00,1.29,3.34,"['shop', 'numbuzin', 'gopicky', 'pickyreview',..."


In [144]:
os.getcwd()

'/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130'

In [146]:
mean_df.to_csv('tiktoker_summary_0131.csv',index=False, encoding="utf-8")

In [147]:
mean_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,tiktoker_size,ad_cost,ER,avg_upload_interval,hash_tag_top5
0,.blissdiaries,"81,700.00","9,717.97","1,447.33",60.47,196.20,micro_influancer,25.00,1.85,4.07,"['kpopfyp', 'douyin', 'haul', 'xyzabc', 'kbeau..."
1,.m.egan,"238,200.00","178,070.00","23,864.30",182.53,"3,527.30",middle_influancer,125.00,10.10,5.03,"['beautyinfluencer', 'jolse', 'xiaohongshu', '..."
2,.woniluv,"135,600.00","41,956.20","2,399.00",85.77,369.93,middle_influancer,125.00,1.83,2.38,"['tiktokban', 'tiktokbanned', 'pretty', 'dance..."
3,4chriisty,"79,500.00","26,122.50","4,119.73",74.13,385.37,micro_influancer,25.00,5.28,2.48,"['jbeauty', 'blue', 'romand', 'pr', 'palette']"
4,__.annie.s,"37,300.00","10,332.73","2,375.10",128.47,465.10,micro_influancer,25.00,6.71,3.14,"['influencer', 'abib', 'influencers', 'minivlo..."
5,_luvwonsnnx,"23,900.00","8,783.47","1,593.63",57.87,261.97,micro_influancer,25.00,6.91,1.48,"['foruu', 'dontletthisflop', 'fyppp', 'viralvi..."
6,aeiezo.pdf,"21,400.00","20,647.13","2,587.73",95.90,350.23,micro_influancer,25.00,12.54,3.48,"['moot', 'creatorsearchinsights', 'routine', '..."
7,ameenbeautytips,"4,800,000.00","418,445.97","1,602.87",14.77,143.10,mega_influancer,"2,500.00",0.03,0.17,['ameenbeautytips']
8,amyflamy1,"4,300,000.00","2,490,070.00","272,720.00","1,466.93","19,117.43",mega_influancer,"2,500.00",6.38,1.45,"['gurwm', 'gurwmroutine', 'grwmroutine', 'make..."
9,asiamelon,"13,700.00","1,751.23",164.30,12.27,19.77,micro_influancer,25.00,1.29,3.34,"['shop', 'numbuzin', 'gopicky', 'pickyreview',..."


In [43]:
os.getcwd()

'/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130'

In [38]:
corr_df = mean_df[["view_cnt","like_cnt","comment_cnt",	"save_cnt",	 "ad_cost",	"ER",	"avg_upload_interval","follower_cnt"]]
corr_df.corr()

,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER,avg_upload_interval,follower_cnt
view_cnt,1.00,0.88,0.60,0.89,0.36,0.39,-0.12,0.47
like_cnt,0.88,1.00,0.76,0.82,0.42,0.20,-0.10,0.56
comment_cnt,0.60,0.76,1.00,0.66,0.33,0.03,-0.13,0.45
save_cnt,0.89,0.82,0.66,1.00,0.31,0.43,-0.12,0.35
ad_cost,0.36,0.42,0.33,0.31,1.00,-0.21,-0.17,0.81
ER,0.39,0.20,0.03,0.43,-0.21,1.00,0.03,-0.18
avg_upload_interval,-0.12,-0.10,-0.13,-0.12,-0.17,0.03,1.00,-0.18
follower_cnt,0.47,0.56,0.45,0.35,0.81,-0.18,-0.18,1.00


In [39]:
mean_df.describe()

,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER,avg_upload_interval
count,56.00,56.00,56.00,56.00,56.00,56.00,56.00,56.00
mean,"470,629.54","246,204.17","20,198.40",174.63,"2,836.41",456.25,12.00,2.80
std,"924,280.71","489,895.19","41,024.61",260.20,"4,444.31",869.31,21.73,3.54
min,"4,121.00","1,690.83",73.27,0.50,0.67,5.00,0.03,0.17
25%,"50,125.00","17,977.23","2,049.13",38.74,331.83,25.00,1.88,1.19
50%,"142,600.00","59,798.30","7,734.62",80.58,920.40,125.00,5.14,2.22
75%,"390,500.00","275,160.12","24,266.07",204.46,"3,025.32",125.00,11.97,3.35
max,"4,800,000.00","2,518,470.00","272,720.00","1,466.93","20,036.77","2,500.00",112.37,25.31


In [40]:
micro_df = mean_df[mean_df['tiktoker_size'] == 'micro_influancer'].reset_index(drop=True)
micro_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,tiktoker_size,ad_cost,ER,avg_upload_interval
0,.blissdiaries,"81,700.00","9,717.97","1,447.33",60.47,196.20,micro_influancer,25.00,1.85,4.07
1,4chriisty,"79,500.00","26,122.50","4,119.73",74.13,385.37,micro_influancer,25.00,5.28,2.48
2,__.annie.s,"37,300.00","10,332.73","2,375.10",128.47,465.10,micro_influancer,25.00,6.71,3.14
3,_luvwonsnnx,"23,900.00","8,783.47","1,593.63",57.87,261.97,micro_influancer,25.00,6.91,1.48
4,aeiezo.pdf,"21,400.00","20,647.13","2,587.73",95.90,350.23,micro_influancer,25.00,12.54,3.48
5,asiamelon,"13,700.00","1,751.23",164.30,12.27,19.77,micro_influancer,25.00,1.29,3.34
6,ayheyt3u,"78,900.00","28,749.60","1,745.63",9.00,265.57,micro_influancer,25.00,2.22,5.76
7,cleanlivingkarly,"25,000.00","6,023.67",223.30,15.63,90.07,micro_influancer,25.00,0.96,0.55
8,emchu_,"72,700.00","39,188.43","4,089.50",33.50,549.13,micro_influancer,25.00,5.67,3.41
9,haruqhxz,"47,000.00","8,141.17","1,210.17",34.27,242.43,micro_influancer,25.00,2.65,5.62


In [41]:
filtered_df = micro_df[(micro_df["ER"] >= 5) & (micro_df['avg_upload_interval'] < 7)].reset_index(drop=True)
filtered_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,tiktoker_size,ad_cost,ER,avg_upload_interval
0,4chriisty,"79,500.00","26,122.50","4,119.73",74.13,385.37,micro_influancer,25.00,5.28,2.48
1,__.annie.s,"37,300.00","10,332.73","2,375.10",128.47,465.10,micro_influancer,25.00,6.71,3.14
2,_luvwonsnnx,"23,900.00","8,783.47","1,593.63",57.87,261.97,micro_influancer,25.00,6.91,1.48
3,aeiezo.pdf,"21,400.00","20,647.13","2,587.73",95.90,350.23,micro_influancer,25.00,12.54,3.48
4,emchu_,"72,700.00","39,188.43","4,089.50",33.50,549.13,micro_influancer,25.00,5.67,3.41
5,its.kaylas,"51,100.00","24,006.93","4,257.87",32.47,874.60,micro_influancer,25.00,8.40,1.31
6,krystallee2222,"65,200.00","392,343.33","35,183.00",120.30,"13,670.83",micro_influancer,25.00,54.15,2.24
7,kwyeona,"29,000.00","18,345.80","3,900.70",317.47,824.53,micro_influancer,25.00,14.55,2.31
8,ofeemi7,"98,700.00","152,066.00","8,523.07",28.90,"2,160.83",micro_influancer,25.00,8.66,1.34
9,thelipstickgirly,"92,900.00","2,518,470.00","104,070.37",319.57,"20,036.77",micro_influancer,25.00,112.37,2.76


In [42]:
얼굴 두분 다 나오고 

krystallee2222 ER 높고 > 피부가 좋아서 신빙성 있음... 
emchu_ 컨셉 

SyntaxError: invalid syntax (1796092301.py, line 1)